In [10]:
from typing import Annotated

from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

In [11]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [12]:
llm = ChatGroq(model_name = "llama3-70b-8192")

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

In [13]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

In [14]:
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [15]:
def run_chatbot():
    config = {"configurable": {"thread_id": "1"}}
    
    print("Chatbot ready! Type 'quit' to exit.")
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ['quit', 'exit']:
            break
        
        print("Bot: ", end="", flush=True)
        
        # Stream the response
        events = graph.stream(
            {"messages": [HumanMessage(content=user_input)]},
            config,
            stream_mode="values"
        )
        
        for event in events:
            if "messages" in event:
                last_message = event["messages"][-1]
                if hasattr(last_message, 'content'):
                    print(last_message.content, end="", flush=True)
        
        print() 
if __name__ == "__main__":
    # Choose one:
    run_chatbot() 

Chatbot ready! Type 'quit' to exit.
Bot: heyyyyyyyyyyyy{"query": "heyyyyyyyyyyyy", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://heyyyyyyyyyyyy.com/", "title": "heyyyyyyyyyyyy.com | email with *nuance*", "content": "Minimalism at its best. heyyyyyyyyyyyy.com is literally just an email address, nothing else. Couldn't be simpler! No ads. No tracking. No promotions.", "score": 0.7892503, "raw_content": null}, {"url": "https://www.youtube.com/watch?v=ZZ5LpwO-An4&pp=0gcJCfwAo7VqN5tD", "title": "HEYYEYAAEYAAAEYAEYAA - YouTube", "content": "MWYAAAH please note I do not own this video.", "score": 0.24211644, "raw_content": null}], "response_time": 1.21}It seems like the tool call yielded a list of search results for the query "heyyyyyyyyyyyy". The results include a few websites and a YouTube video. However, there is no clear answer to the query. It seems like the user might be looking for information or clarification on what "heyyyyyyyyyyyy" is or mean